In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_iris

In [0]:
# path = "/Workspace/sid-v2/regression/data/iris.names"
# data = pd.read_csv(path)

# data.head(6)

In [0]:
iris =  load_iris()

In [0]:
data = pd.DataFrame(iris.data, columns=iris.feature_names)
data.head(6)

In [0]:
data["target"] = iris.target

In [0]:
traindf, testdf = train_test_split(data, test_size=0.25, random_state=406, stratify=data["target"])

In [0]:
xtrain = traindf.drop(columns=["target"])
ytrain = traindf["target"]

xtest = testdf.drop(columns=["target"])
ytest = testdf["target"]


In [0]:
scaler = StandardScaler()
xtrain_scaled = scaler.fit_transform(xtrain)  
xtest_scaled = scaler.transform(xtest)        


xtraintensor = torch.tensor(xtrain_scaled, dtype=torch.float32)
xtesttensor = torch.tensor(xtest_scaled, dtype=torch.float32)

ytraintensor = torch.tensor(ytrain.values, dtype=torch.long)
ytesttensor = torch.tensor(ytest.values, dtype=torch.long)   


print("12")

In [0]:
xtrain.shape

In [0]:
class IrisClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(IrisClassifier, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim),
        )
    
    def forward(self, x):
        return self.network(x)

In [0]:
input_dim = xtrain.shape[1]
hidden_dim = 16
output_dim = 3


In [0]:
model = IrisClassifier(input_dim, hidden_dim, output_dim)

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [0]:
#training

epochs = 500
for epoch in range(epochs):
    model.train()
    optimizer.zero_grad()

    predictions  = model(xtraintensor)
    loss = criterion(predictions, ytraintensor)

    loss.backward()
    optimizer.step()

    if (epoch + 1) % 50 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], loss : {loss.item():.4f}")
